In [0]:
import mlflow
from sklearn.model_selection import train_test_split
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from mlflow.tracking import MlflowClient
import pandas as pd
from sklearn import svm 
from sklearn.metrics import mean_squared_error
import math

# load the datasets 
userhome = 'dbfs:/user/skim658@gwu.edu'

redDF = spark.read.parquet(userhome + '/final-project/reddf.parquet')
whiteDF = spark.read.parquet(userhome + '/final-project/whitedf.parquet')

/databricks/python/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
 from imp import reload

In [0]:
# split the dataset into train and test sets 
redPDDF = redDF.toPandas() 
redQuality = redPDDF['quality']
redPDDF.drop('quality', axis = 1, inplace = True)
redXTrain, redXTest, redYTrain, redYTest = train_test_split(redPDDF, redQuality, test_size = 0.2, random_state = 1)

In [0]:
whitePDDF = whiteDF.toPandas()
whiteQuality = whitePDDF['quality']
whitePDDF.drop('quality', axis = 1, inplace = True)
whiteXTrain, whiteXTest, whiteYTrain, whiteYTest = train_test_split(whitePDDF, whiteQuality, test_size = 0.2, random_state = 1)

## Support Vector Machine (With Hyperopt)

In [0]:
searchSpace = {
  'c': hp.uniform('c', 1, 30),
  'epsilon': hp.uniform('epsilon', 0, 1),
  'kernel': hp.choice('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
}

## Red Wine

In [0]:
def objective_function(params):
  with mlflow.start_run(nested = True):
    c = params['c']
    epsilon = params['epsilon']
    kernel = params['kernel']
    
    # model 
    svmModel = svm.SVR(C = c, epsilon = epsilon, kernel = kernel)
    
    # fit the model 
    svmModel.fit(redXTrain, redYTrain) 
    
    # evaluate predictions 
    redPred = svmModel.predict(redXTest)
    rmse = math.sqrt(mean_squared_error(redYTest, redPred))
    mlflow.log_metric('rmse', rmse)
   
    # log parameters 
    mlflow.log_param('c', c) 
    mlflow.log_param('epsilon', epsilon)
    mlflow.log_param('kernel', kernel)
  
  return {'loss': rmse, 'status': STATUS_OK}

# create parent run 
with mlflow.start_run(run_name = 'RED-SVC-All-Features') as run:
  num_evals = 100 
  trials = Trials()
  best_hyperparam = fmin(fn = objective_function,
                        space = searchSpace,
                        algo = tpe.suggest,
                        max_evals = num_evals,
                        trials = trials)
  
  # log param and metric for best model 
  for name, value in best_hyperparam.items():
    mlflow.log_param(name, value)
  mlflow.log_metric('loss', trials.best_trial['result']['loss'])

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:00<01:22, 1.20trial/s, best loss: 0.7124373237930313] 2%|▏ | 2/100 [00:01<01:16, 1.28trial/s, best loss: 0.6755866560839979] 3%|▎ | 3/100 [00:02<01:12, 1.34trial/s, best loss: 0.6755866560839979] 4%|▍ | 4/100 [00:02<01:07, 1.42trial/s, best loss: 0.6755866560839979] 5%|▌ | 5/100 [00:03<01:14, 1.27trial/s, best loss: 0.6755866560839979] 6%|▌ | 6/100 [00:04<01:12, 1.29trial/s, best loss: 0.6755866560839979] 7%|▋ | 7/100 [00:26<11:12, 7.23s/trial, best loss: 0.6211066431980811] 8%|▊ | 8/100 [00:27<08:15, 5.39s/trial, best loss: 0.6211066431980811] 9%|▉ | 9/100 [00:28<06:00, 3.96s/trial, best loss: 0.6211066431980811] 10%|█ | 10/100 [00:29<04:26, 2.96s/trial, best loss: 0.6211066431980811] 11%|█ | 11/100 [00:29<03:26, 2.32s/trial, best loss: 0.6211066431980811] 12%|█▏ | 12/100 [00:30<02:41, 1.83s/trial, best loss: 0.6211066431980811] 13%|█▎ | 13/100 [00:31<02:19, 1.60s/trial, best loss: 0.6211066431980811] 14%|█▍ | 14/100 [01:23<23:53, 16.67s/trial, best loss: 0.6201680968244552] 15%|█▌ | 15/100 [01:24<16:49, 11.88s/trial, best loss: 0.6201680968244552] 16%|█▌ | 16/100 [01:24<11:56, 8.53s/trial, best loss: 0.6201680968244552] 17%|█▋ | 17/100 [01:25<08:33, 6.18s/trial, best loss: 0.6201680968244552] 18%|█▊ | 18/100 [01:26<06:21, 4.65s/trial, best loss: 0.6201680968244552] 19%|█▉ | 19/100 [01:27<04:47, 3.55s/trial, best loss: 0.6201680968244552] 20%|██ | 20/100 [01:28<03:36, 2.71s/trial, best loss: 0.6201680968244552] 21%|██ | 21/100 [02:10<19:12, 14.59s/trial, best loss: 0.6201506488880045] 22%|██▏ | 22/100 [02:39<24:34, 18.91s/trial, best loss: 0.6201506488880045] 23%|██▎ | 23/100 [04:07<50:50, 39.62s/trial, best loss: 0.6201506488880045] 24%|██▍ | 24/100 [04:46<49:41, 39.24s/trial, best loss: 0.6195171863240652] 25%|██▌ | 25/100 [05:11<44:00, 35.20s/trial, best loss: 0.6195171863240652] 26%|██▌ | 26/100 [05:47<43:26, 35.22s/trial, best loss: 0.6193888816791198] 27%|██▋ | 27/100 [06:41<49:53, 41.00s/trial, best loss: 0.6177887936957958] 28%|██▊ | 28/100 [07:31<52:19, 43.60s/trial, best loss: 0.6177887936957958] 29%|██▉ | 29/100 [07:31<36:22, 30.74s/trial, best loss: 0.6177887936957958] 30%|███ | 30/100 [08:18<41:22, 35.46s/trial, best loss: 0.6177887936957958] 31%|███ | 31/100 [08:39<35:38, 31.00s/trial, best loss: 0.6177887936957958] 32%|███▏ | 32/100 [09:45<47:02, 41.51s/trial, best loss: 0.6177887936957958] 33%|███▎ | 33/100 [09:46<32:47, 29.37s/trial, best loss: 0.6177887936957958] 34%|███▍ | 34/100 [11:20<53:47, 48.91s/trial, best loss: 0.6177887936957958] 35%|███▌ | 35/100 [11:50<46:48, 43.21s/trial, best loss: 0.6177887936957958] 36%|███▌ | 36/100 [12:15<40:21, 37.83s/trial, best loss: 0.6177887936957958] 37%|███▋ | 37/100 [12:16<28:02, 26.70s/trial, best loss: 0.6177887936957958] 38%|███▊ | 38/100 [13:05<34:31, 33.41s/trial, best loss: 0.6177887936957958] 39%|███▉ | 39/100 [14:29<49:24, 48.60s/trial, best loss: 0.6177887936957958] 40%|████ | 40/100 [14:30<34:14, 34.24s/trial, best loss: 0.6177887936957958] 41%|████ | 41/100 [15:56<48:58, 49.81s/trial, best loss: 0.6177887936957958] 42%|████▏ | 42/100 [16:57<51:14, 53.01s/trial, best loss: 0.6177887936957958] 43%|████▎ | 43/100 [16:57<35:28, 37.34s/trial, best loss: 0.6177887936957958] 44%|████▍ | 44/100 [16:59<24:48, 26.59s/trial, best loss: 0.6177887936957958] 45%|████▌ | 45/100 [17:11<20:20, 22.20s/trial, best loss: 0.6177887936957958] 46%|████▌ | 46/100 [17:11<14:10, 15.75s/trial, best loss: 0.6177887936957958] 47%|████▋ | 47/100 [17:12<09:55, 11.23s/trial, best loss: 0.6177887936957958] 48%|████▊ | 48/100 [17:14<07:19, 8.44s/trial, best loss: 0.6177887936957958] 49%|████▉ | 49/100 [18:16<20:54, 24.60s/trial, best loss: 0.6177887936957958] 50%|█████ | 50/100 [18:17<14:31, 17.43s/trial, best loss: 0.6177887936957958] 51%|█████ | 51/100 [18:21<10:51, 13.30s/trial, best loss: 0.6177887936957958] 52%|█████▏ | 52/100 [18:22<07:47, 9.75s/trial, best loss: 0.6177887

In [0]:
if best_hyperparam['kernel'] == 0:
  best_hyperparam['kernel'] = 'linear'
elif best_hyperparam['kernel'] == 1:
  best_hyperparam['kernel'] = 'poly'
elif best_hyperparam['kernel'] == 2:
  best_hyperparam['kernel'] = 'rbf'
else:
  best_hyperparam['kernel'] = 'sigmoid'
redResult = pd.DataFrame(best_hyperparam, index = [0])
display(redResult)

c,epsilon,kernel
19.69920882164071,0.314123751937602,linear


In [0]:
# get best trial rmse 
temp = {}
temp['rmse'] = trials.best_trial['result']['loss']
redMetricsDF = pd.DataFrame(temp, index = [0])
display(redMetricsDF)

rmse
0.6177887936957958


## White Wine

In [0]:
def objective_function(params):
  with mlflow.start_run(nested = True):
    c = params['c']
    epsilon = params['epsilon']
    kernel = params['kernel']
    
    # model 
    svmModel = svm.SVR(C = c, epsilon = epsilon, kernel = kernel)
    
    # fit the model 
    svmModel.fit(whiteXTrain, whiteYTrain) 
    
    # evaluate predictions 
    whitePred = svmModel.predict(whiteXTest) 
    rmse = math.sqrt(mean_squared_error(whiteYTest, whitePred))
    mlflow.log_metric('rmse', rmse) 
   
    # log parameters 
    mlflow.log_param('c', c) 
    mlflow.log_param('epsilon', epsilon) 
    mlflow.log_param('kernel', kernel)
  
  return {'loss': rmse, 'status': STATUS_OK} 

# create parent run  
with mlflow.start_run(run_name = 'WHITE-SVC-All-Features') as run: 
  num_evals = 40   
  trials = Trials() 
  best_hyperparam = fmin(fn = objective_function, 
                        space = searchSpace, 
                        algo = tpe.suggest, 
                        max_evals = num_evals,  
                        trials = trials) 
  
  # log param and metric for best model 
  for name, value in best_hyperparam.items():  
    mlflow.log_param(name, value)
  mlflow.log_metric('loss', trials.best_trial['result']['loss']) 
  
  

0%| | 0/40 [00:00<?, ?trial/s, best loss=?] 2%|▎ | 1/40 [00:21<13:40, 21.05s/trial, best loss: 0.7514701050242498] 5%|▌ | 2/40 [07:10<1:27:07, 137.57s/trial, best loss: 0.7418146147317454] 8%|▊ | 3/40 [07:12<59:48, 96.99s/trial, best loss: 0.7418146147317454]  10%|█ | 4/40 [15:29<2:10:05, 216.81s/trial, best loss: 0.7418146147317454] 12%|█▎ | 5/40 [15:30<1:28:47, 152.20s/trial, best loss: 0.7418146147317454] 15%|█▌ | 6/40 [19:46<1:43:55, 183.40s/trial, best loss: 0.7418146147317454] 18%|█▊ | 7/40 [19:48<1:10:53, 128.89s/trial, best loss: 0.7418146147317454] 20%|██ | 8/40 [28:02<2:07:10, 238.44s/trial, best loss: 0.7418146147317454] 22%|██▎ | 9/40 [28:04<1:26:29, 167.39s/trial, best loss: 0.7390984937315966] 25%|██▌ | 10/40 [28:05<58:46, 117.54s/trial, best loss: 0.7390984937315966]  28%|██▊ | 11/40 [30:54<1:04:21, 133.15s/trial, best loss: 0.7390984937315966] 30%|███ | 12/40 [31:57<52:11, 111.83s/trial, best loss: 0.7390984937315966]  32%|███▎ | 13/40 [31:58<35:25, 78.74s/trial, best loss: 0.7390984937315966]  35%|███▌ | 14/40 [32:00<24:07, 55.69s/trial, best loss: 0.7390984937315966] 38%|███▊ | 15/40 [32:01<16:21, 39.25s/trial, best loss: 0.7390984937315966] 40%|████ | 16/40 [32:03<11:11, 28.00s/trial, best loss: 0.7390984937315966] 42%|████▎ | 17/40 [32:05<07:49, 20.41s/trial, best loss: 0.7390984937315966] 45%|████▌ | 18/40 [32:07<05:24, 14.76s/trial, best loss: 0.7390984937315966] 48%|████▊ | 19/40 [32:08<03:44, 10.69s/trial, best loss: 0.7390984937315966] 50%|█████ | 20/40 [40:44<54:06, 162.32s/trial, best loss: 0.7390984937315966] 52%|█████▎ | 21/40 [40:46<36:11, 114.26s/trial, best loss: 0.7390984937315966] 55%|█████▌ | 22/40 [40:48<24:08, 80.48s/trial, best loss: 0.7389459738525425]  57%|█████▊ | 23/40 [40:50<16:06, 56.84s/trial, best loss: 0.7389459738525425] 60%|██████ | 24/40 [40:51<10:44, 40.30s/trial, best loss: 0.7386924143299984] 62%|██████▎ | 25/40 [40:53<07:10, 28.71s/trial, best loss: 0.7386924143299984] 65%|██████▌ | 26/40 [40:55<04:49, 20.65s/trial, best loss: 0.7386924143299984] 68%|██████▊ | 27/40 [40:57<03:14, 14.98s/trial, best loss: 0.7386924143299984] 70%|███████ | 28/40 [40:58<02:09, 10.77s/trial, best loss: 0.7386924143299984] 72%|███████▎ | 29/40 [40:59<01:28, 8.02s/trial, best loss: 0.7386924143299984] 75%|███████▌ | 30/40 [41:01<01:01, 6.16s/trial, best loss: 0.7386924143299984] 78%|███████▊ | 31/40 [41:03<00:42, 4.74s/trial, best loss: 0.7386924143299984] 80%|████████ | 32/40 [41:04<00:30, 3.77s/trial, best loss: 0.737228438775591]  82%|████████▎ | 33/40 [41:06<00:23, 3.35s/trial, best loss: 0.737228438775591] 85%|████████▌ | 34/40 [41:08<00:17, 2.84s/trial, best loss: 0.737228438775591] 88%|████████▊ | 35/40 [41:10<00:12, 2.57s/trial, best loss: 0.737228438775591] 90%|█████████ | 36/40 [41:13<00:10, 2.74s/trial, best loss: 0.737228438775591] 92%|█████████▎| 37/40 [41:15<00:07, 2.34s/trial, best loss: 0.737228438775591] 95%|█████████▌| 38/40 [41:15<00:03, 1.94s/trial, best loss: 0.737228438775591] 98%|█████████▊| 39/40 [41:18<00:02, 2.21s/trial, best loss: 0.737228438775591]100%|██████████| 40/40 [49:19<00:00, 145.83s/trial, best loss: 0.737228438775591]100%|██████████| 40/40 [49:19<00:00, 73.99s/trial, best loss: 0.737228438775591]

In [0]:
if best_hyperparam['kernel'] == 0:
  best_hyperparam['kernel'] = 'linear'
elif best_hyperparam['kernel'] == 1:
  best_hyperparam['kernel'] = 'poly'
elif best_hyperparam['kernel'] == 2:
  best_hyperparam['kernel'] = 'rbf'
else:
  best_hyperparam['kernel'] = 'sigmoid'
whiteResult = pd.DataFrame(best_hyperparam, index = [0])
display(whiteResult)

c,epsilon,kernel
29.93188577379107,0.29421734533015964,rbf


In [0]:
# get best trial rmse 
temp = {}
temp['rmse'] = trials.best_trial['result']['loss']
whiteMetricsDF = pd.DataFrame(temp, index = [0])
display(whiteMetricsDF)

rmse
0.737228438775591


## Scale

In [0]:
scaler = StandardScaler()
redXTrainS = scaler.fit_transform(redXTrain)
redXTestS = scaler.transform(redXTest) 

## Red Wine

In [0]:
def objective_function(params):
  with mlflow.start_run(nested = True):
    c = params['c']
    epsilon = params['epsilon']
    kernel = params['kernel']
    
    # model 
    svmModel = svm.SVR(C = c, epsilon = epsilon, kernel = kernel)
    
    # fit the model 
    svmModel.fit(redXTrainS, redYTrain)
    
    # evaluate predictions 
    redPred = svmModel.predict(redXTestS)
    rmse = math.sqrt(mean_squared_error(redYTest, redPred))
    mlflow.log_metric('rmse', rmse)
  
    # log parameters 
    mlflow.log_param('c', c)
    mlflow.log_param('epsilon', epsilon)
    mlflow.log_param('kernel', kernel)
  
  return {'loss': rmse, 'status': STATUS_OK}

# create parent run 
with mlflow.start_run(run_name = 'RED-SVC-All-Features') as run:
  num_evals = 200
  trials = Trials()
  best_hyperparam = fmin(fn = objective_function,
                        space = searchSpace,
                        algo = tpe.suggest,
                        max_evals = num_evals,
                        trials = trials)
  
  # log param and metric for best model 
  for name, value in best_hyperparam.items():
    mlflow.log_param(name, value)
  mlflow.log_metric('loss', trials.best_trial['result']['loss'])

0%| | 0/200 [00:00<?, ?trial/s, best loss=?] 0%| | 1/200 [00:00<02:38, 1.25trial/s, best loss: 283.5260419304062] 1%| | 2/200 [00:02<03:38, 1.10s/trial, best loss: 0.6233574011650374] 2%|▏ | 3/200 [00:05<05:43, 1.74s/trial, best loss: 0.6233574011650374] 2%|▏ | 4/200 [00:06<04:48, 1.47s/trial, best loss: 0.6233574011650374] 2%|▎ | 5/200 [00:09<06:22, 1.96s/trial, best loss: 0.6233574011650374] 3%|▎ | 6/200 [00:10<05:09, 1.59s/trial, best loss: 0.6233574011650374] 4%|▎ | 7/200 [00:11<04:19, 1.34s/trial, best loss: 0.6233574011650374] 4%|▍ | 8/200 [00:12<03:45, 1.18s/trial, best loss: 0.6233574011650374] 4%|▍ | 9/200 [00:13<03:44, 1.18s/trial, best loss: 0.6233574011650374] 5%|▌ | 10/200 [00:14<03:20, 1.06s/trial, best loss: 0.6233574011650374] 6%|▌ | 11/200 [00:14<02:55, 1.07trial/s, best loss: 0.6233574011650374] 6%|▌ | 12/200 [00:15<02:42, 1.16trial/s, best loss: 0.6233574011650374] 6%|▋ | 13/200 [00:16<02:41, 1.16trial/s, best loss: 0.6233574011650374] 7%|▋ | 14/200 [00:17<02:42, 1.14trial/s, best loss: 0.6233574011650374] 8%|▊ | 15/200 [00:17<02:33, 1.21trial/s, best loss: 0.6233574011650374] 8%|▊ | 16/200 [00:18<02:28, 1.24trial/s, best loss: 0.6233574011650374] 8%|▊ | 17/200 [00:19<02:22, 1.28trial/s, best loss: 0.6233574011650374] 9%|▉ | 18/200 [00:20<02:17, 1.32trial/s, best loss: 0.6233574011650374] 10%|▉ | 19/200 [00:20<02:19, 1.30trial/s, best loss: 0.6233574011650374] 10%|█ | 20/200 [00:21<02:18, 1.30trial/s, best loss: 0.6233574011650374] 10%|█ | 21/200 [00:22<02:21, 1.27trial/s, best loss: 0.6232553998902074] 11%|█ | 22/200 [00:23<02:52, 1.03trial/s, best loss: 0.6232553998902074] 12%|█▏ | 23/200 [00:24<02:45, 1.07trial/s, best loss: 0.6227340277778484] 12%|█▏ | 24/200 [00:25<02:45, 1.06trial/s, best loss: 0.6227340277778484] 12%|█▎ | 25/200 [00:26<02:39, 1.10trial/s, best loss: 0.6219622498564711] 13%|█▎ | 26/200 [00:27<02:39, 1.09trial/s, best loss: 0.6219622498564711] 14%|█▎ | 27/200 [00:28<02:31, 1.14trial/s, best loss: 0.6219622498564711] 14%|█▍ | 28/200 [00:28<02:21, 1.22trial/s, best loss: 0.6219622498564711] 14%|█▍ | 29/200 [00:29<02:36, 1.10trial/s, best loss: 0.6219622498564711] 15%|█▌ | 30/200 [00:30<02:34, 1.10trial/s, best loss: 0.6219622498564711] 16%|█▌ | 31/200 [00:32<03:16, 1.16s/trial, best loss: 0.6219622498564711] 16%|█▌ | 32/200 [00:33<02:53, 1.03s/trial, best loss: 0.6219622498564711] 16%|█▋ | 33/200 [00:34<03:00, 1.08s/trial, best loss: 0.6219622498564711] 17%|█▋ | 34/200 [00:35<02:54, 1.05s/trial, best loss: 0.6219622498564711] 18%|█▊ | 35/200 [00:37<03:20, 1.21s/trial, best loss: 0.6219622498564711] 18%|█▊ | 36/200 [00:38<03:39, 1.34s/trial, best loss: 0.6219622498564711] 18%|█▊ | 37/200 [00:39<03:13, 1.19s/trial, best loss: 0.6219622498564711] 19%|█▉ | 38/200 [00:40<02:50, 1.06s/trial, best loss: 0.6219622498564711] 20%|█▉ | 39/200 [00:41<02:48, 1.04s/trial, best loss: 0.6219622498564711] 20%|██ | 40/200 [00:42<03:00, 1.13s/trial, best loss: 0.6219622498564711] 20%|██ | 41/200 [00:43<02:39, 1.00s/trial, best loss: 0.6219622498564711] 21%|██ | 42/200 [00:44<02:20, 1.13trial/s, best loss: 0.6219622498564711] 22%|██▏ | 43/200 [00:44<02:10, 1.20trial/s, best loss: 0.6219622498564711] 22%|██▏ | 44/200 [00:45<02:20, 1.11trial/s, best loss: 0.6219622498564711] 22%|██▎ | 45/200 [00:46<02:16, 1.14trial/s, best loss: 0.6219622498564711] 23%|██▎ | 46/200 [00:48<03:15, 1.27s/trial, best loss: 0.6219622498564711] 24%|██▎ | 47/200 [00:49<02:44, 1.08s/trial, best loss: 0.6219622498564711] 24%|██▍ | 48/200 [00:50<02:26, 1.04trial/s, best loss: 0.6219622498564711] 24%|██▍ | 49/200 [00:50<02:10, 1.16trial/s, best loss: 0.6219622498564711] 25%|██▌ | 50/200 [00:51<02:06, 1.19trial/s, best loss: 0.6219622498564711] 26%|██▌ | 51/200 [00:52<02:00, 1.24trial/s, best loss: 0.6219622498564711] 26%|██▌ | 52/200 [00:53<02:09, 1.14trial/s, best loss: 0.6219622498564711] 26%|██▋ | 53/200 [00:53<01:59, 1.23trial/s, best loss: 0.6219622498564711] 27%|██▋ | 54/200 [00

In [0]:
if best_hyperparam['kernel'] == 0:
  best_hyperparam['kernel'] = 'linear'
elif best_hyperparam['kernel'] == 1:
  best_hyperparam['kernel'] = 'poly'
elif best_hyperparam['kernel'] == 2:
  best_hyperparam['kernel'] = 'rbf'
else:
  best_hyperparam['kernel'] = 'sigmoid'
redResult = pd.DataFrame(best_hyperparam, index = [0])
display(redResult)

c,epsilon,kernel
20.967432504027283,0.24831993707176986,linear


In [0]:
# get best trial rmse 
temp = {}
temp['rmse'] = trials.best_trial['result']['loss']
redMetricsDF = pd.DataFrame(temp, index = [0])
display(redMetricsDF)

rmse
0.6207436384472562


## White Wine

In [0]:
scaler = StandardScaler()
whiteXTrainS = scaler.fit_transform(whiteXTrain)
whiteXTestS = scaler.transform(whiteXTest) 

In [0]:
def objective_function(params):
  with mlflow.start_run(nested = True):
    c = params['c']
    epsilon = params['epsilon']
    kernel = params['kernel']
    
    # model 
    svmModel = svm.SVR(C = c, epsilon = epsilon, kernel = kernel)
    
    # fit the model 
    svmModel.fit(whiteXTrainS, whiteYTrain)
    
    # evaluate predictions 
    whitePred = svmModel.predict(whiteXTestS)
    rmse = math.sqrt(mean_squared_error(whiteYTest, whitePred))
    mlflow.log_metric('rmse', rmse)
  
    # log parameters 
    mlflow.log_param('c', c)
    mlflow.log_param('epsilon', epsilon)
    mlflow.log_param('kernel', kernel)
  
  return {'loss': rmse, 'status': STATUS_OK}

# create parent run 
with mlflow.start_run(run_name = 'WHITE-SVC-All-Features') as run:
  num_evals = 200
  trials = Trials()
  best_hyperparam = fmin(fn = objective_function,
                        space = searchSpace,
                        algo = tpe.suggest,
                        max_evals = num_evals,
                        trials = trials)
  
  # log param and metric for best model 
  for name, value in best_hyperparam.items():
    mlflow.log_param(name, value)
  mlflow.log_metric('loss', trials.best_trial['result']['loss'])

0%| | 0/200 [00:00<?, ?trial/s, best loss=?] 0%| | 1/200 [00:01<06:09, 1.86s/trial, best loss: 869.5105361825977] 1%| | 2/200 [00:16<19:10, 5.81s/trial, best loss: 0.8335900778434462] 2%|▏ | 3/200 [00:27<23:55, 7.29s/trial, best loss: 0.7335042038980006] 2%|▏ | 4/200 [00:29<18:27, 5.65s/trial, best loss: 0.7335042038980006] 2%|▎ | 5/200 [00:37<20:47, 6.40s/trial, best loss: 0.7335042038980006] 3%|▎ | 6/200 [00:46<22:48, 7.06s/trial, best loss: 0.7335042038980006] 4%|▎ | 7/200 [00:48<17:38, 5.48s/trial, best loss: 0.7335042038980006] 4%|▍ | 8/200 [00:49<14:01, 4.38s/trial, best loss: 0.6932811047867719] 4%|▍ | 9/200 [00:51<11:32, 3.63s/trial, best loss: 0.6932811047867719] 5%|▌ | 10/200 [00:54<10:56, 3.46s/trial, best loss: 0.6932811047867719] 6%|▌ | 11/200 [00:55<08:36, 2.73s/trial, best loss: 0.6932811047867719] 6%|▌ | 12/200 [00:57<07:14, 2.31s/trial, best loss: 0.6932811047867719] 6%|▋ | 13/200 [00:59<07:29, 2.41s/trial, best loss: 0.6932811047867719] 7%|▋ | 14/200 [01:02<07:21, 2.37s/trial, best loss: 0.6932811047867719] 8%|▊ | 15/200 [01:03<06:55, 2.25s/trial, best loss: 0.6932811047867719] 8%|▊ | 16/200 [01:07<07:39, 2.50s/trial, best loss: 0.6932811047867719] 8%|▊ | 17/200 [01:08<06:20, 2.08s/trial, best loss: 0.6879651068749787] 9%|▉ | 18/200 [01:33<27:27, 9.05s/trial, best loss: 0.6879651068749787] 10%|▉ | 19/200 [01:39<24:51, 8.24s/trial, best loss: 0.6879651068749787] 10%|█ | 20/200 [01:56<32:27, 10.82s/trial, best loss: 0.6879651068749787] 10%|█ | 21/200 [01:59<24:47, 8.31s/trial, best loss: 0.6877236224932043] 11%|█ | 22/200 [02:01<19:29, 6.57s/trial, best loss: 0.6875387507857054] 12%|█▏ | 23/200 [02:04<15:56, 5.40s/trial, best loss: 0.6875387507857054] 12%|█▏ | 24/200 [02:06<13:15, 4.52s/trial, best loss: 0.6875387507857054] 12%|█▎ | 25/200 [02:09<11:20, 3.89s/trial, best loss: 0.6875387507857054] 13%|█▎ | 26/200 [02:13<11:13, 3.87s/trial, best loss: 0.6875387507857054] 14%|█▎ | 27/200 [02:15<09:56, 3.45s/trial, best loss: 0.6875387507857054] 14%|█▍ | 28/200 [02:17<08:42, 3.04s/trial, best loss: 0.6812393041128364] 14%|█▍ | 29/200 [02:19<07:48, 2.74s/trial, best loss: 0.6812393041128364] 15%|█▌ | 30/200 [02:21<07:10, 2.53s/trial, best loss: 0.6812393041128364] 16%|█▌ | 31/200 [02:24<07:31, 2.67s/trial, best loss: 0.6812393041128364] 16%|█▌ | 32/200 [02:26<06:42, 2.40s/trial, best loss: 0.6812393041128364] 16%|█▋ | 33/200 [02:34<11:09, 4.01s/trial, best loss: 0.6812393041128364] 17%|█▋ | 34/200 [02:36<09:25, 3.41s/trial, best loss: 0.6812393041128364] 18%|█▊ | 35/200 [02:39<09:15, 3.37s/trial, best loss: 0.6812393041128364] 18%|█▊ | 36/200 [02:41<07:49, 2.86s/trial, best loss: 0.6812393041128364] 18%|█▊ | 37/200 [02:43<07:17, 2.68s/trial, best loss: 0.6812393041128364] 19%|█▉ | 38/200 [02:45<06:36, 2.44s/trial, best loss: 0.6812393041128364] 20%|█▉ | 39/200 [02:48<07:28, 2.78s/trial, best loss: 0.6812393041128364] 20%|██ | 40/200 [02:50<06:11, 2.32s/trial, best loss: 0.6812393041128364] 20%|██ | 41/200 [02:52<05:54, 2.23s/trial, best loss: 0.6812393041128364] 21%|██ | 42/200 [02:53<05:12, 1.98s/trial, best loss: 0.6812393041128364] 22%|██▏ | 43/200 [03:11<17:29, 6.68s/trial, best loss: 0.6812393041128364] 22%|██▏ | 44/200 [03:14<14:25, 5.55s/trial, best loss: 0.6812393041128364] 22%|██▎ | 45/200 [03:15<11:28, 4.44s/trial, best loss: 0.6812393041128364] 23%|██▎ | 46/200 [03:26<16:06, 6.27s/trial, best loss: 0.6812393041128364] 24%|██▎ | 47/200 [03:33<16:35, 6.51s/trial, best loss: 0.6812393041128364] 24%|██▍ | 48/200 [03:34<12:25, 4.91s/trial, best loss: 0.6812393041128364] 24%|██▍ | 49/200 [03:48<19:19, 7.68s/trial, best loss: 0.6812393041128364] 25%|██▌ | 50/200 [03:50<14:25, 5.77s/trial, best loss: 0.6812393041128364] 26%|██▌ | 51/200 [03:52<11:26, 4.61s/trial, best loss: 0.6812393041128364] 26%|██▌ | 52/200 [03:55<10:47, 4.38s/trial, best loss: 0.6812393041128364] 26%|██▋ | 53/200 [04:07<15:43, 6.42s/trial, best loss: 0.6812393041128364] 27%|██▋ | 54/200 [0

In [0]:
if best_hyperparam['kernel'] == 0:
  best_hyperparam['kernel'] = 'linear'
elif best_hyperparam['kernel'] == 1:
  best_hyperparam['kernel'] = 'poly'
elif best_hyperparam['kernel'] == 2:
  best_hyperparam['kernel'] = 'rbf'
else:
  best_hyperparam['kernel'] = 'sigmoid'
whiteResult = pd.DataFrame(best_hyperparam, index = [0])
display(whiteResult)

c,epsilon,kernel
7.69538546472743,0.47759779607063896,rbf


In [0]:
# get best trial rmse 
temp = {}
temp['rmse'] = trials.best_trial['result']['loss']
whiteMetricsDF = pd.DataFrame(temp, index = [0])
display(whiteMetricsDF)

rmse
0.6783760018076169
